In [100]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from datetime import datetime, timedelta
import pandas as pd
from urllib.request import urlopen
import json

In [101]:
def nameLookup(itemID):
    itemURL = "https://cafemaker.wakingsands.com/item/" + str(itemID)
    itemInfoRaw = urlopen(itemURL).read().decode('utf-8')
    # print(itemInfoRaw)
    itemInfo = json.loads(itemInfoRaw)
    return itemInfo['Name']

In [102]:
# query = { 'itemID': 36258 }

In [103]:
one_day_ago = datetime.utcnow() - timedelta(hours=24)
one_week_ago = datetime.utcnow() - timedelta(days=7)

In [104]:
pipeline = [
    {
        "$match": {
            "lastUploadTime": {"$gte": one_day_ago}
        }
    },
    {
        "$group": {
            "_id": "$itemID",
            "viewCount": {"$sum": 1}
        }
    },
    {
        "$sort": {"viewCount": -1}
    },
    {
        "$limit": 5
    },
    {
        "$project": {
            "itemID": "$_id",
            "viewCount": "$viewCount",
            "_id": 0
        }
    }
]

In [105]:
uri = 'put uri in file mongo_uri'
with open('mongo_uri', 'r') as reader:
    uri = reader.read()
client = MongoClient(uri, server_api=ServerApi('1'))
db = client["universalis"]
collection = db["ws"]

# cursor = collection.find(query, {"_id": 0})
# for i in cursor:
#     print(i)

result = list(collection.aggregate(pipeline))
# print(result)

In [106]:
df = pd.DataFrame(result)
name = []
for i in df['itemID']:
    n = nameLookup(i)
    # print(n)
    name.append(n)
df['name'] = name

In [107]:
df[['itemID', 'name', 'viewCount']]

,itemID,name,viewCount
0,39720,石榴红棉布,6
1,31091,南洋织物,6
2,8841,雇员幻想药,6
3,12537,白钛矿,5
4,39706,软银矿,4
